# DonorChoose.org Kaggle Challenge

Objective: Leverage data provided by DonorChoose.org to devise a recommendation intended to convert first-time donors into repeat donors.

Summary: This notebook provides the data analysis which is the basis of my findings/recommendation. Results of this analysis are communicated through (1) the supporting visualization/dashboard, and (2) the supporting slides which are meant to be read in together with this notebook.

## Table of Contents:

Loading Data into DB
    
<a href = '#helper_functions'> - Helper Functions </a> <br>
<a href = '#csv_loading'> - Loading CSVs into SQLite3 Database </a> <br>
<a href = '#data_cleaning'> - Data Cleaning </a> <br>

Exploratory Data Analysis

<a href = '#eda'> - EDA </a> <br>
<a href = '#preliminary_analysis'> - Preliminary Analysis </a> <br>
<a href = '#donor_location'> - Donor Location </a> <br>
<a href = '#donation_amount'> - Donation Amount </a> <br>
<a href = '#teacher_donors'> - Proportion of Teachers </a> <br>
<a href = '#project_types'> - Repeat Donor Project Types </a> <br>

In [22]:
import sqlite3
import csv
import pandas as pd

<a id = 'helper_functions'></a>
# Making some helper functions
The below assists in the preparation of the database which will house the DonorChoose information, and subsequent operations including the querying of the database.

In [23]:
#Helper function add_table() which takes a doc string to create a table and commit to db
def add_table(docstring):
    conn = sqlite3.connect('donations.db')
    c = conn.cursor()
    c.execute(docstring)
    conn.commit()
    conn.close()

#Helper function import_csv() which takes the name of a csv file (same directory) and imports into specified table
def import_csv(table_name, csv_name, sql_statement):
    conn = sqlite3.connect('donations.db')
    c = conn.cursor()

    with open(csv_name, 'r') as file_:

        reader = csv.reader(file_)

        next(reader)

        for line in reader:
            c.execute(sql_statement, line)

        file_.close()
        conn.commit()
        conn.close

#Helper function return_query() to return output of sql_query
def return_query(query):
    conn = sqlite3.connect('donations.db')
    c = conn.cursor()
    c.execute(query)
    return(c.fetchall())
    conn.commit()
    conn.close()
    
#Helper function drop_table to drop a table

def drop_table(table):
    conn = sqlite3.connect('donations.db')
    c = conn.cursor()
    
    c.execute("DROP TABLE "+table)
    
    conn.commit()
    conn.close()

<a id = 'csv_loading'></a>
# Loading csv files into Sqlite3 db

In [256]:
### Loading the 'donations' table

In [24]:
add_table("""CREATE TABLE donations (
            project_id text,
            donation_id text,
            donor_id text,
            optional_donation text,
            donation_amount real,
            cart_sequence integer,
            receipt_date text
)""")

sql_string = "INSERT INTO donations VALUES(?,?,?,?,?,?,?)"
import_csv('donations', 'Donations.csv', sql_string)

sample_donations = pd.DataFrame(return_query('SELECT * FROM donations LIMIT 3'))
sample_donations

,0,1,2,3,4,5,6
0,000009891526c0ade7180f8423792063,688729120858666221208529ee3fc18e,1f4b5b6e68445c6c4a0509b3aca93f38,No,178.37,11,2016-08-23 13:15:57
1,000009891526c0ade7180f8423792063,dcf1071da3aa3561f91ac689d1f73dee,4aaab6d244bf3599682239ed5591af8a,Yes,25.00,2,2016-06-06 20:05:23
2,000009891526c0ade7180f8423792063,18a234b9d1e538c431761d521ea7799d,0b0765dc9c759adc48a07688ba25e94e,Yes,20.00,3,2016-06-06 14:08:46


In [31]:
### Loading the 'donors' table

In [25]:
add_table("""CREATE TABLE donors (
            donor_id text,
            donor_city text,
            donor_state text,
            donor_is_teacher text,
            donor_zip integer
)""")

sql_string = "INSERT INTO donors VALUES(?,?,?,?,?)"
import_csv('donors', 'Donors.csv', sql_string)

sample_donor = pd.DataFrame(return_query('SELECT * FROM donors LIMIT 3'))
sample_donor

,0,1,2,3,4
0,00000ce845c00cbf0686c992fc369df4,Evanston,Illinois,No,602
1,00002783bc5d108510f3f9666c8b1edd,Appomattox,other,No,245
2,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953


In [32]:
### Loading the 'projects' table

In [26]:
add_table("""CREATE TABLE projects (
            project_id text,
            school_id text,
            teacher_id text,
            posted_sequence integer,
            project_type text,
            project_title text,
            project_essay text,
            short_description text,
            need_statement text,
            category text,
            subcategory text,
            grade_level text,
            project_resource_category text,
            cost real,
            project_posted_date text,
            project_expiration_date text,
            project_status text,
            project_funded_date text
)""")

sql_string = "INSERT INTO projects VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
import_csv('projects', 'Projects.csv', sql_string)

sample_project = pd.DataFrame(return_query('SELECT * FROM projects LIMIT 3'))
sample_project

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,7685f0265a19d7b52a470ee4bac883ba,e180c7424cb9c68cb49f141b092a988f,4ee5200e89d9e2998ec8baad8a3c5968,25,Teacher-Led,Stand Up to Bullying: Together We Can!,Did you know that 1-7 students in grades K-12 ...,Did you know that 1-7 students in grades K-12 ...,"My students need 25 copies of ""Bullying in Sch...",Applied Learning,"Character Education, Early Development",Grades PreK-2,Technology,361.80,2013-01-01,2013-05-30,Fully Funded,2013-01-11
1,f9f4af7099061fb4bf44642a03e5c331,08b20f1e2125103ed7aa17e8d76c71d4,cca2d1d277fb4adb50147b49cdc3b156,3,Teacher-Led,Learning in Color!,"Help us have a fun, interactive listening cent...","Help us have a fun, interactive listening cent...","My students need a listening center, read alon...","Applied Learning, Literacy & Language","Early Development, Literacy",Grades PreK-2,Technology,512.85,2013-01-01,2013-05-31,Expired,
2,afd99a01739ad5557b51b1ba0174e832,1287f5128b1f36bf8434e5705a7cc04d,6c5bd0d4f20547a001628aefd71de89e,1,Teacher-Led,Help Second Grade ESL Students Develop Languag...,Visiting or moving to a new place can be very ...,Visiting or moving to a new place can be very ...,My students need beginning vocabulary audio ca...,Literacy & Language,ESL,Grades PreK-2,Supplies,435.92,2013-01-01,2013-05-30,Fully Funded,2013-05-22


In [33]:
### Loading the 'resources' table

In [27]:
add_table("""CREATE TABLE resources (
            project_id text,
            resource_name text,
            resource_quantity real,
            resource_unit_price real,
            resource_vendor text
)""")

sql_string = "INSERT INTO resources VALUES(?,?,?,?,?)"
import_csv('resources', 'Resources.csv', sql_string)

sample_resource = pd.DataFrame(return_query('SELECT * FROM resources LIMIT 3'))
sample_resource

,0,1,2,3,4
0,000009891526c0ade7180f8423792063,chair move and store cart,1.0,350.00,
1,00000ce845c00cbf0686c992fc369df4,sony mdr zx100 blk headphones,40.0,12.86,CDW-G
2,00002d44003ed46b066607c5455a999a,"gaiam kids stay-n-play balance ball, grey",4.0,19.00,Amazon Business


In [34]:
### Loading the 'schools' table

In [28]:
add_table("""CREATE TABLE schools (
            school_id text,
            school_name text,
            school_metro text,
            school_percentage real,
            school_state text,
            school_zip integer,
            school_city text,
            school_county text,
            school_district text
)""")

sql_string = "INSERT INTO schools VALUES(?,?,?,?,?,?,?,?,?)"
import_csv('schools', 'Schools.csv', sql_string)

sample_school = pd.DataFrame(return_query('SELECT * FROM schools LIMIT 3'))
sample_school

,0,1,2,3,4,5,6,7,8
0,00003e0fdd601b8ea0a6eb44057b9c5e,Capon Bridge Middle School,rural,56.0,West Virginia,26711,Capon Bridge,Hampshire,Hampshire Co School District
1,00004e32a448b4832e1b993500bf0731,The Woodlands College Park High School,urban,41.0,Texas,77384,The Woodlands,Montgomery,Conroe Ind School District
2,0002021bb799f28de224f1acc1ff08c4,Samantha Smith Elementary School,suburban,2.0,Washington,98074,Sammamish,King,Lake Washington Sch Dist 414


In [35]:
### Loading the 'teachers' table

In [29]:
add_table("""CREATE TABLE teachers (
            teacher_id text,
            teacher_prefix text,
            teacher_first_project text
)""")

sql_string = "INSERT INTO teachers VALUES(?,?,?)"
import_csv('teachers', 'Teachers.csv', sql_string)

sample_teacher = pd.DataFrame(return_query('SELECT * FROM teachers LIMIT 3'))
sample_teacher

,0,1,2
0,00000f7264c27ba6fea0c837ed6aa0aa,Mrs.,2013-08-21
1,00002d44003ed46b066607c5455a999a,Mrs.,2016-10-23
2,00006084c3d92d904a22e0a70f5c119a,Mr.,2016-09-08


<a id = 'data_cleaning'></a>
# Data Cleaning

In [36]:
### Before performing EDA towards our objective, I'll quickly check for completeness of each table. 
### Given we have millions of records in our database, we'll investigate any table containing fields that -
### have more than 3% missing data. 

In [58]:
### Check for nulls in the 'donations' table:

donations_fields = ['project_id', 'donation_id', 'donor_id', 'optional_donation', 'donation_amount', 'cart_sequence',
            'receipt_date']

for column in donations_fields:
    num_nulls = return_query("SELECT COUNT(*) FROM donations WHERE "+column+" ='' ")
    num_total = return_query("SELECT COUNT(*) FROM donations")
    percent_null = str(num_nulls[0][0]/num_total[0][0])
    
    print("Percent of "+column+" that are Null"+": "+percent_null)

Percent of project_id that are Null: 0.0
Percent of donation_id that are Null: 0.0
Percent of donor_id that are Null: 0.0
Percent of optional_donation that are Null: 0.0
Percent of donation_amount that are Null: 0.0
Percent of cart_sequence that are Null: 0.0
Percent of receipt_date that are Null: 0.0


In [60]:
## Check for nulls in the 'donors' table:

donors_fields = ['donor_id',
            'donor_city',
            'donor_state',
            'donor_is_teacher',
            'donor_zip']

for column in donors_fields:
    num_nulls = return_query("SELECT COUNT(*) FROM donors WHERE "+column+" = '' ")
    num_total = return_query("SELECT COUNT(*) FROM donors")
    percent_null = str(num_nulls[0][0] / num_total[0][0])
    
    print("Percent of "+column+" that are Null"+": "+percent_null)

Percent of donor_id that are Null: 0.0
Percent of donor_city that are Null: 0.10039243583462104
Percent of donor_state that are Null: 0.0
Percent of donor_is_teacher that are Null: 0.0
Percent of donor_zip that are Null: 0.08482832698903253


In [59]:
## Check for nulls in the 'projects' table:

projects_fields = ['project_id',
            'school_id',
            'teacher_id',
            'posted_sequence',
            'project_type',
            'project_title',
            'project_essay',
            'short_description',
            'need_statement',
            'category',
            'subcategory',
            'grade_level',
            'project_resource_category',
            'cost',
            'project_posted_date',
            'project_expiration_date',
            'project_status',
            'project_funded_date']

for column in projects_fields:
    num_nulls = return_query("SELECT COUNT(*) FROM projects WHERE "+column+" = '' ")
    num_total = return_query("SELECT COUNT(*) FROM projects")
    percent_null = str(num_nulls[0][0] / num_total[0][0])
    
    print("Percent of "+column+" that are Null"+": "+percent_null)

Percent of project_id that are Null: 0.0
Percent of school_id that are Null: 0.0
Percent of teacher_id that are Null: 0.0
Percent of posted_sequence that are Null: 0.0
Percent of project_type that are Null: 0.0
Percent of project_title that are Null: 5.405322621185081e-06
Percent of project_essay that are Null: 9.008871035308468e-07
Percent of short_description that are Null: 2.7026613105925407e-06
Percent of need_statement that are Null: 2.7026613105925407e-06
Percent of category that are Null: 2.612572600239456e-05
Percent of subcategory that are Null: 2.612572600239456e-05
Percent of grade_level that are Null: 0.0
Percent of project_resource_category that are Null: 3.243193572711048e-05
Percent of cost that are Null: 0.0
Percent of project_posted_date that are Null: 0.0
Percent of project_expiration_date that are Null: 1.2612419449431855e-05
Percent of project_status that are Null: 0.0
Percent of project_funded_date that are Null: 0.255178974736423


In [61]:
## Check for nulls in the 'resources' table:

resources_fields = ['project_id',
            'resource_name',
            'resource_quantity',
            'resource_unit_price',
            'resource_vendor']

for column in resources_fields:
    num_nulls = return_query("SELECT COUNT(*) FROM resources WHERE "+column+" = '' ")
    num_total = return_query("SELECT COUNT(*) FROM resources")
    percent_null = str(num_nulls[0][0] / num_total[0][0])
    
    print("Percent of "+column+" that are Null"+": "+percent_null)

Percent of project_id that are Null: 0.0
Percent of resource_name that are Null: 7.780376475913841e-05
Percent of resource_quantity that are Null: 0.0033699709088811126
Percent of resource_unit_price that are Null: 0.0033714964728959974
Percent of resource_vendor that are Null: 0.011371554166953287


In [62]:
## Check for nulls in the 'schools' table:

schools_fields = ['school_id',
            'school_name',
            'school_metro',
            'school_percentage',
            'school_state',
            'school_zip',
            'school_city',
            'school_county',
            'school_district']

for column in schools_fields:
    num_nulls = return_query("SELECT COUNT(*) FROM schools WHERE "+column+" = '' ")
    num_total = return_query("SELECT COUNT(*) FROM schools")
    percent_null = str(num_nulls[0][0] / num_total[0][0])
    
    print("Percent of "+column+" that are Null"+": "+percent_null)

Percent of school_id that are Null: 0.0
Percent of school_name that are Null: 0.0
Percent of school_metro that are Null: 0.0
Percent of school_percentage that are Null: 0.015631635910292766
Percent of school_state that are Null: 0.0
Percent of school_zip that are Null: 0.0
Percent of school_city that are Null: 0.003109887249462277
Percent of school_county that are Null: 2.739988766046059e-05
Percent of school_district that are Null: 0.0


In [63]:
## Check for nulls in the 'teachers' table:

teachers_fields = ['teacher_id',
            'teacher_prefix',
            'teacher_first_project']

for column in teachers_fields:
    num_nulls = return_query("SELECT COUNT(*) FROM teachers WHERE "+column+" = '' ")
    num_total = return_query("SELECT COUNT(*) FROM teachers")
    percent_null = str(num_nulls[0][0] / num_total[0][0])
    
    print("Percent of "+column+" that are Null"+": "+percent_null)

Percent of teacher_id that are Null: 0.0
Percent of teacher_prefix that are Null: 6.949615289153636e-05
Percent of teacher_first_project that are Null: 0.0


### Conclusion on completeness of data:

In our databse, only one table - the 'donor' table - contains a significant (i.e., greater than 3% per field) amount of omissions. In particular, the 'donor_city' and 'donor_zip' (zipcode of donor) fields have significant proportion of missing data. Our subsequent analysis will not require these fields as input and therefore no further work considered necessary.

<a id = "eda"></a>
# Exploratory Data Analysis

<a id = "preliminary_analysis"></a>
# Preliminary Analysis

In [68]:
### Caluclating the count, mean and sum of all donations

count = str(round(return_query('SELECT COUNT(*) FROM donations')[0][0],2))
mean = str(round(return_query('SELECT AVG(donation_amount) FROM donations')[0][0],2))
total = str(round(return_query('SELECT SUM(donation_amount) FROM donations')[0][0],2))

print("Summary of donations:")
print("count: {}".format(count))
print("mean: {}".format(mean))
print("sum: {}".format(total))


Summary of donations:
count: 4687884
mean: 60.67
sum: 284408243.28


In [73]:
### Calculating the total number of distinct donors and distinct projects

count_donors = str(return_query("SELECT COUNT(DISTINCT donor_id) FROM donors")[0][0])
count_projects = str(return_query("SELECT COUNT(DISTINCT project_id) FROM projects")[0][0])

print("Count of unique donors: {}. Count of unique projects: {}".format(count_donors, count_projects))

Count of unique donors: 2122640. Count of unique projects: 1110015


The above metrics provide a basic understanding of the scale of the information that we're working with. We now move onto more substantial analysis towards our objective. The overall approach in the subsequent analysis attemps to first identify the most important characteristics of repeat donors (that is: donors that have more than a single donation). These characteristics will then be used to filter first-time donors to identify those first-time donors that are susceptible to further donations if targetted with promotional activities. 

In identifying the characteristics of repeat donors, we'll ask the following:

1. Where do repeat donors live?
2. How much do repeat donors tend to donate?
3. Do repeat donors tend to be teachers?
4. What do repeat donors invest in? 

<a id="donor_location"></a>
# Location of repeat donors

Because the 'donor' table provides locations for each donor, we'll visualize the whereabouts of repeat donors in Tableau. To expedite the loading of data into Tableau, we'll output a CSV containing a tablue summary of the location of donors and their donations for those donors that made more than one donation.

## Locating repeat donors

In [83]:
### Before we make our output, let's determine the portion of donors that make repeat donations.

In [79]:
count_repeaters = """
SELECT COUNT(*)
FROM
(SELECT donor_id, COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)>1)
"""

count_firsts = """
SELECT COUNT(*)
FROM
(SELECT donor_id, COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)=1)
"""

count_control = """
SELECT COUNT(DISTINCT donor_id)
FROM donations

"""

repeats = str(return_query(count_repeaters)[0][0])
firsts = str(return_query(count_firsts)[0][0])
control = str(return_query(count_control)[0][0])

print("repeats = {}, firsts = {}, control = {}".format(repeats, firsts, control))


repeats = 552941, firsts = 1471613, control = 2024554


In [82]:
print("Out of a total of {} unique donors, only {} donors made more than 1 donation".format(control, repeats))

Out of a total of 2024554 unique donors, only 552941 donors made more than 1 donation


In [84]:
### Now, for purposes of visualization using Tableau, let's output:

# 1 - a table showing count of repeat donors by state, and;
# 2 - a table showing count of first-time donors by state.

In [121]:
### Outputting two tables showing (1) count of repeat donors by state and (1) count offirst-time donors by state

repeats_query = """
SELECT donors.donor_state, COUNT(*) AS repeat_donors
FROM donors
JOIN
(SELECT donor_id, COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)>1) AS b
ON donors.donor_id = b.donor_id
GROUP BY donors.donor_state"""

single_query = """
SELECT donors.donor_state, COUNT(*) AS firsts_donors
FROM donors
JOIN
(SELECT donor_id, COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)=1) AS b
ON donors.donor_id = b.donor_id
GROUP BY donors.donor_state"""

returns_output = pd.DataFrame(return_query(repeats_query), columns=['State','Repeat Donors'])
single_output = pd.DataFrame(return_query(single_query), columns = ['State', 'First Donors'])

In [122]:
returns_output.head(2)

,State,Repeat Donors
0,Alabama,6046
1,Alaska,1245


In [123]:
single_output.head(2)

,State,First Donors
0,Alabama,16030
1,Alaska,3006


In [125]:
### publishing the two tables to the same directory:

returns_output.to_csv('return_donors_by_state.csv')
single_output.to_csv('first_donors_by_state.csv')

Using the outputs created above, the locations of repeat donors will be visualized in Tableau

<a id = "donation_amount"></a>
## Determining amount typically donated by repeat donors

In [127]:
### In this part of the analysis, we'll identify:

# 1. The mean donation of repeat donors,
# 2. The median donation of repeat donors, and;
# 3. The visual representation of the distribution of donation amounts for all repeat donors.


In [134]:
### Calculating the mean and median donations by repeat donors:

mean_median_query = """
SELECT donation_id, donations.donor_id, donation_amount
FROM donations JOIN
(SELECT donor_id, COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)>1) AS b
ON donations.donor_id = b.donor_id
"""

mean_median_df = pd.DataFrame(return_query(mean_median_query), columns= ['donation_id', 
                                                                        'donor_id',
                                                                        'donation_amount'])

In [147]:
repeat_mean = round(mean_median_df['donation_amount'].mean(), 2)
repeat_median = round(mean_median_df['donation_amount'].median(), 2)

In [158]:
print('The mean donation of repeat donors is {}, and the median donation of repeat donors is {}'\
      .format(repeat_mean, repeat_median))


The mean donation of repeat donors is 63.99, and the median donation of repeat donors is 25.0


It looks like a few large donations are causing a difference in the mean and median donations. To verify, we'll visualize the distribution of donations in Tableau. 

In [161]:
### Output a CSV including donation amounts for all donations made by repeat donors

mean_median_df.to_csv('repeat_donation_amounts.csv')

The distribution of donation amounts for repeat donors is visualized in the accompanying Tableau dashboard.

<a id = 'teacher_donors'></a>
# Do repeat donors tend to be teachers?

In [165]:
# We require a table showing, for all repeat donors, whether or not they are teachers:

teacher_query = """
SELECT donors.donor_id, donors.donor_is_teacher
FROM donors
JOIN
(SELECT donor_id, COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)>1) AS b
ON donors.donor_id = b.donor_id
"""


teacher_df = pd.DataFrame(return_query(teacher_query))

In [212]:
### For visualization purposes, we'll output teacher_df as a CSV file for input into Tableau:

teacher_df[1:].to_csv("repeat_teachers.csv")

In [194]:
are_teachers = teacher_df[1]=='Yes'
are_teachers = are_teachers.sum()

not_teachers = teacher_df[1]=='No'
not_teachers = not_teachers.sum()

percent_repeat_teachers = round(100*are_teachers / (are_teachers + not_teachers),2)

teacher_output = "Of all repeat donors, {} are teachers and {} are not teachers. {} percent of repeat\
                    donors are teachers".format\
                    (str(are_teachers),str(not_teachers), str(percent_repeat_teachers))
print(teacher_output)



Of all repeat donors, 133074 are teachers and 419598 are not teachers. 24.08 percent of repeat                    donors are teachers


In [196]:
### We'll repeat the same analysis for non-repeat donors (i.e., determine the frequency of teachers amongst)
### non-repeat donors.

teacher_query_non_repeat = """
SELECT donors.donor_id, donors.donor_is_teacher
FROM donors
JOIN
(SELECT donor_id, COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)=1) AS b
ON donors.donor_id = b.donor_id
"""

teacher_df_non_repeat = pd.DataFrame(return_query(teacher_query_non_repeat))

In [213]:
### For visualization purposes, we'll output teacher_df as a CSV file for input into Tableau:

teacher_df_non_repeat[1:].to_csv("non_repeat_teachers.csv")

In [210]:
non_repeat_teachers = teacher_df_non_repeat[1] == 'Yes'
non_repeat_teachers = non_repeat_teachers.sum()

non_repeat_regulars = teacher_df_non_repeat[1] == "No"
non_repeat_regulars = non_repeat_regulars.sum()

percent_teachers_firsts = round(100*non_repeat_teachers / (non_repeat_teachers+non_repeat_regulars), 2)

firsts_teacher_output = """Of all first-time donors, {} are teachers and {} are not teachers. {} 
percent of first-time donors are teachers"""

firsts_teacher_output = firsts_teacher_output.format(str(non_repeat_teachers),str(non_repeat_regulars), str(percent_teachers_firsts))
                                                     
print(firsts_teacher_output)



Of all first-time donors, 77165 are teachers and 1389125 are not teachers. 5.26 
percent of first-time donors are teachers


In [239]:
slide_string_1 = "There are a total of {} donors that are repeat donors".format(str(are_teachers+not_teachers))
print(slide_string_1)

There are a total of 552672 donors that are repeat donors


In [216]:
teacher_df_non_repeat.head(2)

,0,1
0,00000ce845c00cbf0686c992fc369df4,No
1,00002783bc5d108510f3f9666c8b1edd,No


The accompanying Tableau Dashboards take as input the above output CSVs and visualize the proportion of teachers amongst first-time and repeat donors.

<a id = "project_types"></a>
# What do repeat donors tend to donate for?

In [251]:
### To identify the common themes that attract repeat donors, we'll create a word cloud in tableau which will
### require as input a table of all the project titles that were donated to by repeat donors. 
### We produce that CSV in the below:

project_query = """

SELECT projects.category, projects.project_posted_date
FROM projects
JOIN
(SELECT donations.project_id, b.donor_id
FROM donations
JOIN
(SELECT donor_id,COUNT(*)
FROM donations
GROUP BY donor_id
HAVING COUNT(*)>1) AS b
ON donations.donor_id = b.donor_id) AS c
ON projects.project_id = c.project_id


"""

returned_projects = return_query(project_query)

We now split each of the names of popular projects into their individual words. we'll output this result as a csv to be visualized in Tableau as a word cloud (to identify popular themes). We'll only consider those projects that were posted in 2018 (so that we can get a sense of which projects were relevant to recent donors).

In [252]:
words = []

for i in returned_projects:
    if i[1][:4] == '2018':
        for j in i[0].split(' '):
            words.append(j)
    else:
        pass

In [255]:
words_df = pd.DataFrame(words)
words_df.to_csv('repeat_donor_projects_words.csv')

In [266]:
my_tables = return_query('SELECT * FROM sqlite_master')
for table in my_tables:
    my_text = table[-1].split("CREATE TABLE")
    my_text = my_text[1]
    print(my_text)
#     print(table[-1])

 donations (
            project_id text,
            donation_id text,
            donor_id text,
            optional_donation text,
            donation_amount real,
            cart_sequence integer,
            receipt_date text
)
 donors (
            donor_id text,
            donor_city text,
            donor_state text,
            donor_is_teacher text,
            donor_zip integer
)
 projects (
            project_id text,
            school_id text,
            teacher_id text,
            posted_sequence integer,
            project_type text,
            project_title text,
            project_essay text,
            short_description text,
            need_statement text,
            category text,
            subcategory text,
            grade_level text,
            project_resource_category text,
            cost real,
            project_posted_date text,
            project_expiration_date text,
            project_status text,
            project_funded_date tex